In [0]:
%sql
SELECT *
FROM gizmobox.bronze.v_customers
WHERE customer_id is not NULL

In [0]:
%sql
SELECT DISTINCT *
FROM gizmobox.bronze.v_customers
WHERE customer_id is not NULL

In [0]:
%sql
create or replace temporary view v_customers_distinct as
SELECT DISTINCT *
FROM gizmobox.bronze.v_customers
WHERE customer_id is not NULL
ORDER BY customer_id

In [0]:
%sql
select customer_id,
MAX(created_timestamp) as max_created_timestamp
from v_customers_distinct
group by customer_id;

In [0]:
%sql
WITH cte_max as (
    select customer_id,
MAX(created_timestamp) as max_created_timestamp
from v_customers_distinct
group by customer_id
)

select *
from v_customers_distinct 
join cte_max
on v_customers_distinct.customer_id = cte_max.customer_id
and  v_customers_distinct.created_timestamp = cte_max.max_created_timestamp

In [0]:
%sql
WITH cte_max as (
    select customer_id,
MAX(created_timestamp) as max_created_timestamp
from v_customers_distinct
group by customer_id
)

select 
CAST(t.created_timestamp as timestamp) as created_timestamp,
t.customer_id,
t.customer_name,
CAST(t.date_of_birth as date) as date_of_birth,
t.email,
CAST(t.member_since as date) as member_since,
t.telephone

from v_customers_distinct t
join cte_max
on t.customer_id = cte_max.customer_id
and  t.created_timestamp = cte_max.max_created_timestamp

In [0]:
%sql
CREATE TABLE gizmobox.silver.customers
AS
WITH cte_max as (
    select customer_id,
MAX(created_timestamp) as max_created_timestamp
from v_customers_distinct
group by customer_id
)

select 
CAST(t.created_timestamp as timestamp) as created_timestamp,
t.customer_id,
t.customer_name,
CAST(t.date_of_birth as date) as date_of_birth,
t.email,
CAST(t.member_since as date) as member_since,
t.telephone

from v_customers_distinct t
join cte_max
on t.customer_id = cte_max.customer_id
and  t.created_timestamp = cte_max.max_created_timestamp

In [0]:
%sql
select *
from gizmobox.silver.customers

In [0]:
%sql
DESCRIBE EXTENDED gizmobox.silver.customers

In [0]:
%sql
create or replace table gizmobox.silver.payments
as

select
payment_id,
cast(date_format(payment_timestamp, 'yyyy-MM-dd') as date) as payment_date,
date_format(payment_timestamp, 'HH:mm:ss') as payment_time,
case 
when payment_status = 1 then 'Success'
when payment_status = 2 then 'Pending'
when payment_status = 3 then 'Cancelled'
when payment_status = 4 then 'Failed'
else 'Unknown'
end as payment_status,
payment_method
from gizmobox.bronze.payments


In [0]:
%sql
create schema hive_metastore.silver;

create or replace table hive_metastore.silver.refunds
as

select
refund_id,
payment_id,
cast(date_format(refund_timestamp, 'yyyy-MM-dd') as date) as refund_date,
date_format(refund_timestamp, 'HH:mm:ss') as refund_time,
refund_amount,
split(refund_reason,':')[0] as refund_reason,
split(refund_reason,':')[1] as reason_source
from hive_metastore.bronze.refunds

In [0]:
%sql
select *
 from hive_metastore.silver.refunds

In [0]:
%sql

create or replace table gizmobox.silver.memberships
as
SELECT 
regexp_extract(path,'.*/([0-9]+)\\.png$',1) AS customer_id,
content
FROM gizmobox.bronze.v_memberships;



In [0]:
%sql
select *
from gizmobox.silver.memberships

In [0]:
%sql
create or replace table gizmobox.silver.addresses
as

SELECT *
FROM (
  select 
  customer_id,
  address_type,
  address_line_1,
  city,
  state,
  postcode
  from gizmobox.bronze.v_addresses
)
pivot (MAX(address_line_1) as address_line_1,
      MAX(city) as city,
      MAX(state) as state,
      MAX(postcode) as postcode
 for address_type in ('shipping','billing'));

In [0]:
%sql
select * from gizmobox.silver.addresses

In [0]:
%sql
SELECT  value: items[0].item_id::INTEGER as item_1,
        value: items[1] as item_2,
        value
FROM gizmobox.bronze.v_orders;

In [0]:
%sql
CREATE or REPLACE TEMP view tv_orders_fixed
as
SELECT
  value,
  regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"') as fixed_value
FROM gizmobox.bronze.v_orders;

In [0]:
%sql
select schema_of_json(fixed_value) as schema,
fixed_value
from tv_orders_fixed

In [0]:
%sql
create table if not exists gizmobox.silver.orders_json
as
select from_json(fixed_value, 'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') as json_value,
fixed_value
from tv_orders_fixed

In [0]:
%sql
select * 
from gizmobox.silver.orders_json

In [0]:
%sql
create or replace temp view tv_orders_exploded
as
select 
json_value.order_id as order_id,
json_value.order_status as order_status,
json_value.order_date as order_date,
json_value.total_amount as total_amount,
json_value.payment_method as payment_method,
json_value.customer_id as customer_id,
explode(array_distinct(json_value.items)) as item
from gizmobox.silver.orders_json

In [0]:
%sql
select
    order_id,
    order_status,
    order_date,
    total_amount,
    payment_method,
    customer_id,
    item.item_id,
    item.name,
    item.price,
    item.quantity,
    item.category,
    item.details.brand,
    item.details.color
from tv_orders_exploded

In [0]:
%sql
create table if not exists gizmobox.silver.orders
as
select
    order_id,
    order_status,
    order_date,
    total_amount,
    payment_method,
    customer_id,
    item.item_id,
    item.name,
    item.price,
    item.quantity,
    item.category,
    item.details.brand,
    item.details.color
from tv_orders_exploded